# 2 Create Mobility Demand

The goal of the following notebook is to compute a realistic Mobility Demand.
___
The Mobility Demand $D = \{T_1, \dots, T_N\}$ is a multiset of $N$ trips (one per each vehicle) within an urban environment. 
A single trip $T_v=(o,d)$ for a vehicle $v$ is defined by its origin location $o$ and destination location $d$.
To compute $D$, we use an origin-destination matrix $M$ where an element $m_{o, d}\in M$ describes the number of vehicles' trips that start in tile $o$ and end in tile $d$. 
Then, we iterate $N$ times the following procedure. 
- A vehicle's $v$ trip is a pair $T_v=(e_o, e_d)$ generated by selecting at random a matrix element $m_{o,d} \in M$ with a probability $p_{o, d} \propto m_{o, d}$ and uniformly at random two edges $e_o, e_d \in E$ within tiles $o$ and $d$, respectively.
___

In [1]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import geopandas as gpd
import sumolib
from utils import *
import json
from itertools import groupby

#### File paths

In [2]:
# od matrix path
od_matrix_path = "../data/od_matrix.npy"

# dict tile-edges path
dict_tile_edges_path = "../data/dict_tile_edges.json"

# output dict mobility demand filename
dict_mobility_demand_path = "../data/dict_mobility_demand.json"

# road network path
road_network_path = "../sumo_simulation_data/milan_road_network.net.xml"

# output filename
mobility_demand_save_path = "../sumo_simulation_data/mobility_demands/"

#### Load the OD matrix

In [3]:
od_matrix = np.load(od_matrix_path)

#### Load the dict tile-edges

In [4]:
with open(dict_tile_edges_path, 'r') as f:
    dict_tile_edges = json.load(f)

#### Load the road network

In [5]:
road_network = sumolib.net.readNet(road_network_path, withInternal=False)

#### Create the Mobility Demand

In [6]:
n_veichles = 20
filename = mobility_demand_save_path+"mobility_demand"

In [7]:
od_pairs, departure_times, dict_mobility_demand = create_mobility_demand(n_veichles, dict_tile_edges, od_matrix, 
                                                                         road_network, filename, timing="uniform_range", 
                                                                         time_range=(0, 3601))

print("Created the mobility demand: "+mobility_demand_save_path+"mobility_demand.rou.xml")

100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [00:03<00:00,  5.10it/s]

Created the mobility demand: ../sumo_simulation_data/mobility_demands/mobility_demand.rou.xml


In [8]:
# save the dictionary that contains for each vehicle its corresponding trip (edge_start, edge_destination) and 
# the associated departure time

output_file = open(dict_mobility_demand_path, "w")
json.dump(dict_mobility_demand, output_file)
output_file.close()